In [138]:
from z3 import *
import time

In [128]:
class Sudoku:
    def __init__(self,sudoku_str):
        self.sudoku_str = sudoku_str

    def get_char(self,row,column):
        return self.sudoku_str[row*9 + column]
    
    def __repr__(self):
        output = ""
        for row in range(0,9):
            for column in range(0,9):
                c = self.sudoku_str[row*9 + column]
                output += str(c)
                if column in [2,5]:
                    output += "|"
            output += "\n"
            if row in [2,5]:
                output+="-"*11 + "\n"
        return output

In [129]:
def load_sudokus(filename : str):
    with open(filename,"r") as infile:
        sudokus = []
        for line in infile.readlines():
            sudoku = []
            for char in line.strip():
                if not (char in ["0", ".", "_"]):
                    sudoku.append(int(char))
                else:
                    sudoku.append(" ")
            sudokus.append(Sudoku(sudoku))

    return sudokus
            
sudokus = load_sudokus("easy50.txt")        

In [141]:
def solve_sudoku(sudoku : Sudoku):
    solver = Solver()
    X = [[ Int("x_%s_%s" % (row, column)) for row in range(9) ] for column in range(9) ]

    # Each digit must be 1 <= x <= 9
    for row in range(9):
        for column in range(9):
            solver.add(And(X[row][column] >= 1, X[row][column] <= 9))
            
    # Add initial numbers of sudoku puzzle
    for row in range(9):
        for column in range(9):
            digit = sudoku.get_char(row,column)
            if digit != " ":
                solver.add(X[row][column] == digit)

    # Each row must contain distinct numbers
    for row in range(9):
        row = X[row]
        solver.add(Distinct(row))

    # Each column must contain distinct numbers
    for column in range(9):
        column = [X[row][column] for row in range(9)]
        solver.add(Distinct(column))
        
    #  Each 3x3 Cell must contain distinct numbers
    for x in range(3):
        for y in range(3):
            cell = []
            for row in range(x*3,(x+1)*3):
                for column in range(y*3,(y+1)*3):
                    cell.append(X[row][column])
            solver.add(Distinct(cell))
                    
            
    if solver.check() == sat:
        m = solver.model()
        output = ""
        for j in range(9):
            for i in range(9):
                output+=str(m.evaluate(X[j][i]))
       
        return Sudoku(output)
    print("unsat")
    return None
            

In [145]:
sudokus = load_sudokus("easy50.txt")        

t0 = time.time()
for sudoku in sudokus:
    solve_sudoku(sudoku)
t1 = time.time()
total = t1-t0

print(f"Solved {len(sudokus)} sudokus in {total}s.")

Solved 50 sudokus in 1.6290171146392822 s.
